In [ ]:
#@title
%%html
<iframe src="https://www.polskacyfrowa.gov.pl/media/48246/FE_POPC_poziom_pl-1_rgb.jpg" width="800"></iframe>


# Uczenie głębokie

Szymon Zaporowski, Politechnika Gdańska, Wydział ETI, Katedra Systemów Multimedialnych

**Wykład 9:** Rekurencyjne Sieci Neuronowe i ich zastosowania w NLP

**Przykład (2):** Przetwarzanie Sequence to Seqence na przykładzie tłumaczenia maszynowego


## Wstęp

Ten przykład pokazuje, jak zaimplementować podstawowy
powtarzalny model sekwencja-do-sekwencji na poziomie znaków.
W tym przypadku stosujemy model do tłumaczenia
krótkich fraz w jezyku angielskim na jeżyk francuski,
znak po znaku. Należy mieć na uwadzę fakt, że standardowo do tłumaczenia maszynowego stosuje się modele na poziomie wyrazów, niemniej jednak na potrzeby prezentacji zastosowano taki model.

**Jak działa poniższy algorytm**

- Zaczynamy od sekwencji wejściowych z domeny (np. frazy w j.angielskim)
    i odpowiadającej sekwencji docelowej z innej domeny
    (np. frazy w j. francuskim).
- Enkoder LSTM zamienia sekwencje wejściowe na 2 wektory stanu
    (zachowujemy ostatni stan LSTM i odrzucamy wyjścia).
- Dekoder LSTM jest trenowany, aby przekształcić sekwencje docelowe w
    tę samą sekwencję, ale przesunięta o jeden krok w przód,
    w tym kontekście wykorzystywany jest proces treningowy  zwany „forsowaniem nauczyciela”.
    Wykorzystuje jako stan początkowy wektory stanu z kodera.
    W efekcie dekoder uczy się generować `targets[t+1...]`
    podane `targets[...t]`, uwarunkowane sekwencją wejściową.
- W trybie wnioskowania (inferencja), gdy chcemy zdekodować nieznane sekwencje wejściowe:
    - Zakodujemy sekwencję wejściową do wektorów stanu
    - Rozpoczniemy od sekwencji docelowej o rozmiarze 1
        (zawirającej tylko znak początku sekwencji)
    - Podamy wektory stanu i 1-znakową sekwencję docelową
        do dekodera, aby uzyskać prognozy dla następnego znaku
    - Próbkujemy następną postać, korzystając z tych predykcji
        (po prostu używamy argmax).
    - Dołączamy próbkowany znak do sekwencji docelowej
    - Powtarzamy, aż wygenerujemy znak końca sekwencji lub
        osiągniemy limit znaków.

Dokonajmy importu potrzebnych bibliotek


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras


Pobierzmy niezbędne dane - przykłądowe frazy w jezyku francuski i angielskim połączone w pary.

In [ ]:
!!curl -O http://www.manythings.org/anki/fra-eng.zip
!!unzip fra-eng.zip


['Archive:  fra-eng.zip',
 '  inflating: _about.txt              ',
 '  inflating: fra.txt                 ']

Zdefinujmy hiperparametry oraz scieżkę do danych


In [ ]:
batch_size = 64  # Batch size dla treningu
epochs = 100  #
latent_dim = 256  # Rozmiar przestrzeni utajonej potrzebnej do kodowania
num_samples = 10000  # Liczba próbek do treningu
# Ścieżka do danych w formacie txt
data_path = "fra.txt"


Dokonajmy przetworzenia danych


In [ ]:
# Wektoryzacja danych .
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # Wykorzystujemy "tab" jako znak początku sekwencji dla sekwencji dolceowych i znak nowej lini "\n" jako znak końca sekwencji
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Liczba próbek:", len(input_texts))
print("Liczba unikalnych wejściowych tokenów:", num_encoder_tokens)
print("Liczba unikalnych wyjściowych tokenów:", num_decoder_tokens)
print("Maksymalna długość sekwencji dla wejść:", max_encoder_seq_length)
print("Maksymalna długość sekwencji dla wyjść:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data  wyprzedza zmienną decoder_input_data o jeden krok czasowy
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data będzie o jeden krok czasowy szybciej
            #  i nie zostanie zawarty w znaku początkowym..
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0


Liczba próbek: 10000
Liczba unikalnych wejściowych tokenów: 71
Liczba unikalnych wyjściowych tokenów: 93
Maksymalna długość sekwencji dla wejść: 15
Maksymalna długość sekwencji dla wyjść: 59


Zbudujmy model zawierający koder i dekoder

In [ ]:
# Definicja sekwencji wejściowej i jej dalsze przetwarzanie
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

#Usuwamy wyjścia kodera (encoder_outputs) i zostawiamy tylko jego stany
encoder_states = [state_h, state_c]

# Ustawiamy dekoder, wykorzystujemy `encoder_states` jako stan początkowy
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# Ustawiamy dekoder, aby zwracał pełną sekwencje wyjściową oraz wewnętrzne stany.
# W przypadku treningu nie wykorzsystujemy stanów powrotnych (return_states),
# ale przydają się one przy generowaniu
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Definujemy model, któr będzie zamieniał `encoder_input_data` i `decoder_input_data`
# w  `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


Teraz dokonajmy treningu modelu


In [ ]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Zapiszmy model
model.save("s2s")


Epoch 1/100
125/125 [==============================] - 10s 19ms/step - loss: 1.1303 - accuracy: 0.7368 - val_loss: 1.0059 - val_accuracy: 0.7125
Epoch 2/100
125/125 [==============================] - 1s 11ms/step - loss: 0.8194 - accuracy: 0.7806 - val_loss: 1.0299 - val_accuracy: 0.7433
Epoch 3/100
125/125 [==============================] - 1s 11ms/step - loss: 0.6572 - accuracy: 0.8161 - val_loss: 0.7202 - val_accuracy: 0.7919
Epoch 4/100
125/125 [==============================] - 1s 11ms/step - loss: 0.5954 - accuracy: 0.8289 - val_loss: 0.6500 - val_accuracy: 0.8106
Epoch 5/100
125/125 [==============================] - 1s 11ms/step - loss: 0.5348 - accuracy: 0.8434 - val_loss: 0.6136 - val_accuracy: 0.8201
Epoch 6/100
125/125 [==============================] - 1s 11ms/step - loss: 0.4993 - accuracy: 0.8534 - val_loss: 0.5662 - val_accuracy: 0.8357
Epoch 7/100
125/125 [==============================] - 1s 11ms/step - loss: 0.4707 - accuracy: 0.8612 - val_loss: 0.5513 - val_accuracy

## Inferencja - próbkowanie

1.   Dokonamy zakodowanie wejścia i pobrania stanu pozątkowego dekodera
2.   Przesuniemy się o jeden krok w dekoderze za pomocą stanu początkowego i ze tokenem "startu sekwencji" jako targetu. Wyjście będzie następnym tokenem targetu (sekwencji docelowej)
3. Powtórzmy z kolejnymi docelowymi targetami i obecnymi stanami



In [ ]:
# Definicja modeli próbkujących
# Przywracamy model z wcześniej zapisanych plików, wraz z checkpointem
# tworzym koder i dekoder
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # wejście 1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  #  1 warstwa lstm
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # wejscie 2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

#Powrotna zamiana tokenów, aby zdekodowane sewkencje dało się odczytać
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Zakodowanie wejścia jako wektora stanu
    states_value = encoder_model.predict(input_seq)

    # Tworzenie pustej sekwencji docelowej o długości 1
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Zapełnienie pierwszego znaku sekwencji dolecowej poprzez wpisanie znaku rozpoczynającego
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Pętla próbkująca do stwworzenia batcha sekwencji (uproszczone założenie, że batch ma rozmiar 1)
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Próbkowanie tokenu
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Warunek zakończenia: osiągnienie maksymalnej długości lub znalezienie znaku stopu
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Aktualizacja doceliowej sekwencji
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Aktualizacja stanu
        states_value = [h, c]
    return decoded_sentence



Spróbujmy wygenerować frazy bazując na wytrenowanym modelu:


In [ ]:
for seq_index in range(100):
    # Bierzemy jedną sekwencję (część zbioru treningowego), aby wypróbować dekodowanie
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Wejściowa fraza:", input_texts[seq_index])
    print("Zdekodowana fraza:", decoded_sentence)


1/1 [==============================] - 0s 14ms/step
-
Wejściowa fraza: Go.
Zdekodowana fraza: Va !

1/1 [==============================] - 0s 16ms/step
-
Wejściowa fraza: Go.
Zdekodowana fraza: Va !

1/1 [==============================] - 0s 21ms/step
-
Wejściowa fraza: Go.
Zdekodowana fraza: Va !

1/1 [==============================] - 0s 17ms/step
-
Wejściowa fraza: Go.
Zdekodowana fraza: Va !

1/1 [==============================] - 0s 15ms/step
-
Wejściowa fraza: Hi.
Zdekodowana fraza: Salut.

1/1 [==============================] - 0s 28ms/step
-
Wejściowa fraza: Hi.
Zdekodowana fraza: Salut.

1/1 [==============================] - 0s 23ms/step
-
Wejściowa fraza: Run!
Zdekodowana fraza: File !

1/1 [==============================] - 0s 30ms/step
-
Wejściowa fraza: Run!
Zdekodowana fraza: File !

1/1 [==============================] - 0s 33ms/step
-
Wejściowa fraza: Run!
Zdekodowana fraza: File !

1/1 [==============================] - 0s 28ms/step
-
Wejściowa fraza: Run!
Zdekodowana

Zachęcam do eksperymentowania z notatnikiem, prób zwiększania liczby warstw i liczby neuronów.

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>